In [1]:
!pip install crewai
!pip install crewai-tools
!pip install langchain_community

In [2]:
!pip install PyPDF2
!pip install python-docx

In [3]:
#Warning controls
import warnings
warnings.filterwarnings('ignore')

In [4]:
with open(".env", "w") as f:
    f.write("SERPER_API_KEY=f210a2ee7408cb0afb2b91b865744e5d4f6c2c93\n")
    f.write("OPENROUTER_API_KEY=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")

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

SERPER_API_KEY = os.getenv("SERPER_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [6]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
import PyPDF2   # For reading PDF files
import docx     # For reading DOCX files
import json
from typing import Dict, List

In [7]:
from crewai_tools import FileReadTool, SerperDevTool

file_read_tool=FileReadTool()
serper_dev_tool=SerperDevTool()

class SentimentAnalysisTool(BaseTool):
    name:str = "SentimentAnalysisTool"
    description:str=("Analyzes the sentiment of text to ensure positive and engaging communication.")

    def _run(self,text:str)->str:
        return "positive"

sentiment_analysis_tool=SentimentAnalysisTool()

C:\Users\saket.khopkar\anaconda3\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
C:\Users\saket.khopkar\anaconda3\Lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
C:\Users\saket.khopkar\anaconda3\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:35: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@f

In [8]:
job_description_parser_agent=Agent(
    role="Job Description Parser",
    goal="",
    backstory=(
        "You are an AI-powered job description parser. "
        "Your role is to analyze job descriptions and extract critical information"
        "such as required skills, preferred qualifications, experience levels, and key responsibilities. "
        "You provide a clear and structured breakdown of what the job entails, enabling accurate matching with candidate profiles."
    ),
    allow_delegation=False,
    llm='openrouter/deepseek/deepseek-chat-v3-0324:free',
    verbose=True
)

resume_parser_agent=Agent(
    role="Resume Parser",
    goal=(
        "Extract detailed information from resumes, including the candidate's name, skills,"
      "experience, education, certifications, and contact details (email and phone number)."),
    backstory=(
        "You are an AI-powered resume parser with a deep understanding of professional profiles. "
        "Your expertise lies in analyzing resumes and extracting structured data such as the candidate's name, skills, work experience, education, and contact information. "
        "You ensure that no detail is overlooked, providing a comprehensive profile of each candidate for further evaluation."
    ),
    allow_delegation=False,
    llm='openrouter/deepseek/deepseek-chat-v3-0324:free',
    verbose=True
)

skill_matcher_agent= Agent(
    role="Skill Matcher",
    goal="Match candidate skills to job requirements and generate a compatibility score.",
    backstory=(
        "You are an AI-powered skill matcher. "
        "Your expertise lies in comparing the skills and qualifications of candidates with the"
        "requirements of job descriptions. You use advanced algorithms to calculate a compatibility score,"
        "ensuring that only the most suitable candidates are shortlisted for further consideration."
    ),
    allow_delegation=False,
    llm='openrouter/deepseek/deepseek-chat-v3-0324:free',
    verbose=True
)

candidate_outreach_agent= Agent(
    role="Candidate Outreaching",
    goal="Generate personalized and professional emails for candidate outreach, including the candidate's name, fixed job description details, and assigned interview slot.",
    backstory=(
        "You are an AI-powered email generator. "
        "Your role is to craft personalized and compelling emails for candidates who meet the job requirements. "
        "You highlight their skills and experience, explain why they are a great fit for the role, and include the"
        " candidate's name, fixed job description details, and assigned interview slot that is in future(in March, 2025). "
        "Your emails are professional, engaging, and strictly follow the provided template without adding extra information."
    ),
    allow_delegation=False,
    llm='openrouter/deepseek/deepseek-chat-v3-0324:free',
    verbose=True
)

In [9]:
job_description_parsing_task=Task(
    description=(
        "Parse the job description ({job_description}) and extract key requirements, including"
        "required skills, preferred skills, experience levels, and education requirements. "
        "**Do not add any extra information that is not explicitly mentioned in the job-description.** "
        "This task is essential for understanding the job's demands and ensuring accurate matching with candidate profiles."
    ),
    expected_output=(
        "A JSON object containing the job's required skills, preferred skills, experience requirements,"
        "education requirements, and key responsibilities."
    ),
    agent=job_description_parser_agent
)

resume_parsing_task = Task(
    description=(
        "Parse the candidate's resume ({resume_text}) and extract detailed information, including the"
        "candidate's name, skills, experience, education, certifications, and contact details (email and phone number). "
        "**Do not add any extra information that is not explicitly mentioned in the resume.** "
        "This task is critical for building a comprehensive profile of the candidate, which will be used for skill matching and outreach."
    ),
    expected_output=(
        "A JSON object containing the candidate's name, skills, experience, education, certifications, and contact details (email and phone number). "
        "**Do not include any additional fields or information that is not explicitly present in the resume.**"
    ),
    agent=resume_parser_agent
)

skill_matching_task = Task(
    description=(
        "Compare the candidate's skills and qualifications with the job requirements and generate a compatibility score. "
        "**Do not add any extra information that is not explicitly mentioned in the resume.** "
        "This task ensures that only candidates who meet the job's criteria are shortlisted for further consideration."
    ),
    expected_output=(
        "A compatibility score (0 to 1) indicating how well the candidate's skills match the job requirements."
    ),
    agent=skill_matcher_agent
)

candidate_outreach_task = Task(
    description=(
        "Generate a personalized and professional email for the candidate if their compatibility score is above the threshold. "
        "The email should include:\n"
        "1. The candidate's name (extracted from their resume).\n"
        "2. Fixed job description details (required skills, preferred skills, experience levels, education requirements, and key responsibilities).\n"
        "3. The company's details (name, address, contact email, and phone number).\n"
        "4. A call-to-action to confirm the assigned interview slot( which should be in march, 2025).\n"
        "5. A personalized message highlighting the candidate's relevant skills and experience.\n"
        "**Do not add any extra information that is not explicitly provided in the inputs.**"
    ),
    expected_output=(
        "A personalized email draft tailored to the candidate, including:\n"
        "1. The candidate's name from the resume.\n"
        "2. Fixed job description details.\n"
        "3. The company's details.\n"
        "4. The assigned interview slot in March 2025.\n"
        "5. A personalized message and call-to-action.\n"
        "**Do not include any additional information or fields beyond what is explicitly requested.**"
    ),
    tools=[sentiment_analysis_tool],
    agent=candidate_outreach_agent
)

In [10]:
#Function to define file reading function

def read_file(file_path):
  if file_path.lower().endswith('.pdf'):
    return read_pdf(file_path)
  elif file_path.lower().endswith('.docx'):
    return read_docx(file_path)
  else:
    raise ValueError(f"Unsupported file type: {file_path}")

def read_pdf(file_path):
  with open(file_path,'rb') as file:
    reader=PyPDF2.PdfReader(file)
    text=""
    for page_num in range (len(reader.pages)):
      page=reader.pages[page_num]
      text+=page.extract_text() if page.extract_text() else ""
    return text

def read_docx(file_path):
  doc=docx.Document (file_path)
  text=""
  for para in doc.paragraphs:
    text+=para.text+"\n"
  return text

In [11]:
from crewai import Crew
hr_crew=Crew(
    agents=[
        job_description_parser_agent,
        resume_parser_agent,
        skill_matcher_agent,
        candidate_outreach_agent
    ],
    tasks=[
        job_description_parsing_task,
        resume_parsing_task,
        skill_matching_task,
        candidate_outreach_task
    ],
    verbose=True,
    # memory=True   # OPEN AI KEY IS NEEDED FOR THIS - OR ELSE VALIDATION ERROR
)

In [12]:
if __name__=="__main__":
  #File paths for job description and resumes
  jd_file_path=input("Please enter path to job description: ")
  resume_file_paths=input("Please enter paths to resumes (separated by commas): ").split(',')

  #Read job description
  job_description=read_file(jd_file_path)

  #Process each Resume
  selected_candidates=[]
  unselected_candidates=[]

  for resume_file_path in resume_file_paths:
    #Read Resume
    resume_text=read_file(resume_file_path.strip())

    #Run the crew with inputs
    result = hr_crew.kickoff(
            inputs={
                "job_description": job_description,
                "resume_text": resume_text
            }
        )

    #Ensure result is aprsed as JSON if it is a string
    if isinstance(result,str):
      try:
        result_dict = json.loads(result)
      except json.JSONDecodeError:
        print(f"Error decoding json from result: {result}")
        continue

    elif isinstance(result, dict):
      result_dict=result

    else:
      print(f"Unexpected result type: {type(result)}")
      continue

     # Categorize candidates based on compatibility score
    compatibility_score = result_dict.get("compatibility_score", 0)
    if compatibility_score >= 0.7:  # Threshold for selection
      selected_candidates.append(result_dict)
    else:
      unselected_candidates.append(result_dict)

#Printing Results
print("\n Selected Candidates")
for candidate in selected_candidates:
  print(f"Candidate: {candidate.get('candidate_name', 'N/A')}")
  print(f"Compatibility Score: {candidate.get('compatibility_score', 'N/A')}")
  print(f"Email_draft : {candidate.get('email_draft', 'N/A')}")

print("\n Unselected Candidates")
for candidate in selected_candidates:
  print(f"Candidate: {candidate.get('candidate_name', 'N/A')}")
  print(f"Compatibility Score: {candidate.get('compatibility_score', 'N/A')}")

Please enter path to job description:  JD-NodeJS.pdf
Please enter paths to resumes (separated by commas):  Saket_Khopkar_New_CV.pdf, Node-JS-Resume.pdf


# Agent: Job Description Parser
## Task: Parse the job description (Page 1 of 2 
 
 
  JOB DESCRIPTION 
 
Job Title:  NodeJS Developer  
  
Department:  Technology Integration 
 
Location:  Invera Westmount  
 
Reports To:  Yaroslav Afanasiev   
  
  
                  Role Summary/Purpose  
 
Embark on an exciting journey with Invera Inc., 
a company dedicated to creating cutting -edge 
solutions for the metal industry.  We are looking for a skilled Node.js Developer to take on a significant role within our vibrant development team and contribute to a variety of internal and customer- facing projects.  
 In this role, you will leverage your experience and talent as a Node.js Developer to play a crucial role in developing internal tools essential for the growth and efficiency of our company.  You will collaborate with other developers, designers, and key st akeholders to 
deliver high- quality code and solutions. APPLY NOW!  
 
Send your resume to 
careers@invera.com 
Job Responsibilit